In [1]:
import mwclient
import time
import transformers

site = mwclient.Site('en.wikipedia.org')
page = site.pages['Binance']

In [2]:
revs = list(page.revisions())

In [4]:
revs[0]

OrderedDict([('revid', 1229889274),
             ('parentid', 1227647510),
             ('user', 'InternetArchiveBot'),
             ('timestamp',
              time.struct_time(tm_year=2024, tm_mon=6, tm_mday=19, tm_hour=7, tm_min=27, tm_sec=17, tm_wday=2, tm_yday=171, tm_isdst=-1)),
             ('comment',
              'Rescuing 2 sources and tagging 0 as dead.) #IABot (v2.0.9.5) ([[User:Noxoug1|Noxoug1]] - 20085')])

In [5]:
revs = sorted(revs, key=lambda rev: rev["timestamp"]) 

In [6]:
revs[0]

OrderedDict([('revid', 817963944),
             ('parentid', 0),
             ('user', 'El fo0'),
             ('timestamp',
              time.struct_time(tm_year=2017, tm_mon=12, tm_mday=31, tm_hour=17, tm_min=7, tm_sec=33, tm_wday=6, tm_yday=365, tm_isdst=-1)),
             ('comment',
              "[[WP:AES|←]]Created page with '{{Infobox company | name = Binance | foundation = 2016 | location = [[Hong Kong]], China | products = [[Cryptocurrency exchange]] | homepage = [https://www.binan...'")])

In [7]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.MessageMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.ScalarMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\SUBRAT\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to forc

In [8]:
edits = {}

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [9]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

In [10]:
import pandas as pd
edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [11]:
edits_df

,edit_count,sentiment,neg_sentiment
2017-12-31,4,0.315109,0.250000
2018-01-04,1,-0.998359,1.000000
2018-01-15,1,-0.997948,1.000000
2018-01-21,1,-0.999722,1.000000
2018-01-22,1,-0.999730,1.000000
...,...,...,...
2024-05-19,2,-0.013685,0.500000
2024-05-24,1,-0.956601,1.000000
2024-05-31,3,0.287008,0.333333
2024-06-07,1,-0.999601,1.000000


In [12]:
edits_df.index = pd.to_datetime(edits_df.index)
from datetime import datetime
dates = pd.date_range(start="2009-03-08",end=datetime.today())
edits_df = edits_df.reindex(dates, fill_value=0)
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,0,0.000000,0.0
2009-03-09,0,0.000000,0.0
2009-03-10,0,0.000000,0.0
2009-03-11,0,0.000000,0.0
2009-03-12,0,0.000000,0.0
...,...,...,...
2024-06-15,0,0.000000,0.0
2024-06-16,0,0.000000,0.0
2024-06-17,0,0.000000,0.0
2024-06-18,0,0.000000,0.0


In [13]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()
rolling_edits = rolling_edits.dropna()
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-04-06,0.000000,0.000000,0.000000
2009-04-07,0.000000,0.000000,0.000000
2009-04-08,0.000000,0.000000,0.000000
2009-04-09,0.000000,0.000000,0.000000
2009-04-10,0.000000,0.000000,0.000000
...,...,...,...
2024-06-15,0.233333,-0.056096,0.094444
2024-06-16,0.233333,-0.056096,0.094444
2024-06-17,0.233333,-0.056096,0.094444
2024-06-18,0.166667,-0.055640,0.077778


In [14]:
rolling_edits.to_csv("binance_sentiment.csv")